<a href="https://colab.research.google.com/github/ghitadahibi/SRS/blob/main/Copy_of_pcatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sentence-transformers

In [ ]:
pip install openai

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import openai
import time
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
from openpyxl import load_workbook
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
path='clas1.xlsx'
jobs = pd.read_excel(path,engine='openpyxl')
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_id            11 non-null     int64 
 1   job_title         11 non-null     object
 2   job_description   11 non-null     object
 3   skills            11 non-null     object
 4   majors            11 non-null     object
 5   years_experience  11 non-null     object
dtypes: int64(1), object(5)
memory usage: 656.0+ bytes


In [ ]:
jobs.head()

,job_id,job_title,job_description,skills,majors,years_experience
0,0,Data Scientist,data scientist location albuquerque,"[engineering, business, data analysis]","['computer science', 'data analysis']",five year experienced
1,1,Data Analyst,data analyst position in chicago,"[statistics, data visualization, sql, python]","['statistics', 'data analysis']",two year experienced data analyst with advance...
2,2,Data Engineer,senior data engineer position in san francisco,"[big data, data modeling, spark, hadoop, java]","['computer science', 'big data']",seven year experienced data engineer with expe...
3,3,Machine Learning,Engineer machine learning engineer job in new ...,"[machine learning, deep learning, tensorflow, ...","['computer science', 'machine learning']",four year experienced machine learning enginee...
4,4,Data Scientist,data scientist role in boston,"[statistics, machine learning, python, data vi...","['statistics', 'data analysis']",three year experienced data scientist with exp...


In [ ]:
path='new.xlsx'
resumes = pd.read_excel(path,engine='openpyxl')
resumes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   resume_id  11 non-null     int64 
 1   contact    11 non-null     object
 2   skills     11 non-null     object
 3   degrees    11 non-null     object
 4   majors     11 non-null     object
dtypes: int64(1), object(4)
memory usage: 568.0+ bytes


In [ ]:
resumes.head()

,resume_id,contact,skills,degrees,majors
0,0,Jazirimayssa20@mail.com,"[redux,react,dotnet,apache_hadoop,apache_spark]","[BS_LEVEL,MS_LEVEL]","[software engineering, cybersecurity]"
1,1,fatma.guidara@insat.com,"[react,react_native,django,computer_vision]",[BS_LEVEL],"[artificial intelligence, software engineering]"
2,2,shatha.siala@gmail.com,"[react,react_native,dotnet,database,communicat...","[BS_LEVEL,MS_LEVEL]",[software development]
3,3,bouzidi.ilyes2020@gmail.com,"[engineering,sql,machine_learning,c]",[MS LEVEL],[artificial intelligence]
4,4,melek.elloumi@insat.ucar.tn,"[opencv,csharp,deeplearning,java,matlab]",[BS_LEVEL],[Datascience]


In [ ]:
jobs_df = pd.read_excel('clas1.xlsx', engine='openpyxl')
resumes_df = pd.read_excel('new.xlsx', engine='openpyxl')

**Model SBERT all-mpnet-base-v2**

In [ ]:

def semantic_similarity_sbert_base_v2(job, resume):
    """calculate similarity with SBERT all-mpnet-base-v2"""
    model = SentenceTransformer('all-mpnet-base-v2')
    #Encoding:
    score = 0
    sen = job + resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            similarity_scores = cosine_similarity([sen_embeddings[i]], sen_embeddings[len(job):])[0]
            if max(similarity_scores) >= 0.4:
                score += max(similarity_scores)
    score = score / len(job)
    return round(score, 3)

jobs_df = pd.read_excel('clas1.xlsx', engine='openpyxl')
resumes_df = pd.read_excel('new.xlsx', engine='openpyxl')

columns = ['resume_id', 'job_id', 'all-mpnet-base-v2_score']
matching_dataframe = pd.DataFrame(columns=columns)

for job_index in tqdm(range(jobs_df.shape[0])):
    for resume_index in range(resumes_df.shape[0]):
        job_title = jobs_df.loc[job_index, 'job_title']
        job_description = jobs_df.loc[job_index, 'job_description']
        job_skills = jobs_df.loc[job_index, 'skills']
        resume_skills = resumes_df.loc[resume_index, 'skills']
        job_phrases = [sentence.strip() for sentence in (job_title + " " + job_description).split('.') if sentence.strip()]
        score = semantic_similarity_sbert_base_v2(job_skills.split(",") + job_phrases, [resume_skills])
        matching_dataframe = matching_dataframe.append({
            'resume_id': resume_index,
            'job_id': job_index,
            'all-mpnet-base-v2_score': score
        }, ignore_index=True)

matching_dataframe.to_excel('matching_dataset.xlsx', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]<ipython-input-47-1064ff6dbdc5>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_dataframe = matching_dataframe.append({
<ipython-input-47-1064ff6dbdc5>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_dataframe = matching_dataframe.append({
<ipython-input-47-1064ff6dbdc5>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_dataframe = matching_dataframe.append({
<ipython-input-47-1064ff6dbdc5>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_dataframe = matching_dataframe.append({
<ipython-input-47-1064ff6dbdc5>:32: FutureWarning: The frame.append method is depr

In [ ]:
matching_dataframe

,resume_id,job_id,all-mpnet-base-v2_score
0,0.0,0.0,0.000
1,1.0,0.0,0.000
2,2.0,0.0,0.000
3,3.0,0.0,0.123
4,4.0,0.0,0.000
...,...,...,...
116,6.0,10.0,0.000
117,7.0,10.0,0.000
118,8.0,10.0,0.090
119,9.0,10.0,0.000


 **SBERT all-MiniLM-L12-v1**

In [ ]:

def semantic_similarity_sbert_base_v2(job, resume):
    """calculate similarity with SBERT all-MiniLM-L12-v1"""
    model = SentenceTransformer('all-MiniLM-L12-v1')
    #Encoding:
    score = 0
    sen = job + resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            similarity_scores = cosine_similarity([sen_embeddings[i]], sen_embeddings[len(job):])[0]
            if max(similarity_scores) >= 0.4:
                score += max(similarity_scores)
    score = score / len(job)
    return round(score, 3)

jobs_df = pd.read_excel('clas1.xlsx', engine='openpyxl')
resumes_df = pd.read_excel('new.xlsx', engine='openpyxl')

columns = ['resume_id', 'job_id', 'all-MiniLM-L12-v1_score']
matching_dataframe = pd.DataFrame(columns=columns)

for job_index in tqdm(range(jobs_df.shape[0])):
    for resume_index in range(resumes_df.shape[0]):
        job_title = jobs_df.loc[job_index, 'job_title']
        job_description = jobs_df.loc[job_index, 'job_description']
        job_skills = jobs_df.loc[job_index, 'skills']
        resume_skills = resumes_df.loc[resume_index, 'skills']
        job_phrases = [sentence.strip() for sentence in (job_title + " " + job_description).split('.') if sentence.strip()]
        score = semantic_similarity_sbert_base_v2(job_skills.split(",") + job_phrases, [resume_skills])
        matching_dataframe = matching_dataframe.append({
            'resume_id': resume_index,
            'job_id': job_index,
            'all-MiniLM-L12-v1_score': score
        }, ignore_index=True)

matching_dataframe.to_excel('matching_dataset1.xlsx', index=False)

In [ ]:
matching_dataframe

,resume_id,job_id,all-MiniLM-L12-v1_score
0,0.0,0.0,0.000
1,1.0,0.0,0.000
2,2.0,0.0,0.000
3,3.0,0.0,0.117
4,4.0,0.0,0.000
...,...,...,...
116,6.0,10.0,0.000
117,7.0,10.0,0.000
118,8.0,10.0,0.256
119,9.0,10.0,0.000


 **Model all-roberta-large-v1**

In [ ]:
def semantic_similarity_sbert_roberta_large_v1(job, resume):
    """calculate similarity with SBERT all-roberta-large-v1"""
    model = SentenceTransformer('all-roberta-large-v1')
    #Encoding:
    score = 0
    sen = job + resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            similarity_scores = cosine_similarity([sen_embeddings[i]], sen_embeddings[len(job):])[0]
            if max(similarity_scores) >= 0.4:
                score += max(similarity_scores)
    score = score / len(job)
    return round(score, 3)

jobs_df = pd.read_excel('clas1.xlsx', engine='openpyxl')
resumes_df = pd.read_excel('new.xlsx', engine='openpyxl')

columns = ['resume_id', 'job_id', 'all-roberta-large-v1_score']
matching_dataframe = pd.DataFrame(columns=columns)

for job_index in tqdm(range(jobs_df.shape[0])):
    for resume_index in range(resumes_df.shape[0]):
        job_title = jobs_df.loc[job_index, 'job_title']
        job_description = jobs_df.loc[job_index, 'job_description']
        job_skills = jobs_df.loc[job_index, 'skills']
        resume_skills = resumes_df.loc[resume_index, 'skills']
        job_phrases = [sentence.strip() for sentence in (job_title + " " + job_description).split('.') if sentence.strip()]
        score = semantic_similarity_sbert_roberta_large_v1(job_skills.split(",") + job_phrases, [resume_skills])
        matching_dataframe = matching_dataframe.append({
            'resume_id': resume_index,
            'job_id': job_index,
            'all-roberta-large-v1_score': score
        }, ignore_index=True)

matching_dataframe.to_excel('matching_dataset2.xlsx', index=False)

In [ ]:
matching_dataframe

,resume_id,job_id,all-roberta-large-v1_score
0,0.0,0.0,0.000
1,1.0,0.0,0.000
2,2.0,0.0,0.000
3,3.0,0.0,0.245
4,4.0,0.0,0.000
...,...,...,...
116,6.0,10.0,0.000
117,7.0,10.0,0.169
118,8.0,10.0,0.370
119,9.0,10.0,0.000


**Model paraphrase-MiniLM-L6-v2**

In [ ]:

def semantic_similarity_sbert_paraphrase_minilm_l6_v2(job, resume):
    """calculate similarity with SBERT paraphrase-MiniLM-L6-v2"""
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    #Encoding:
    score = 0
    sen = job + resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            similarity_scores = cosine_similarity([sen_embeddings[i]], sen_embeddings[len(job):])[0]
            if max(similarity_scores) >= 0.4:
                score += max(similarity_scores)
    score = score / len(job)
    return round(score, 3)

jobs_df = pd.read_excel('clas1.xlsx', engine='openpyxl')
resumes_df = pd.read_excel('new.xlsx', engine='openpyxl')

columns = ['resume_id', 'job_id', 'paraphrase-MiniLM-L6-v2_score']
matching_dataframe = pd.DataFrame(columns=columns)

for job_index in tqdm(range(jobs_df.shape[0])):
    for resume_index in range(resumes_df.shape[0]):
        job_title = jobs_df.loc[job_index, 'job_title']
        job_description = jobs_df.loc[job_index, 'job_description']
        job_skills = jobs_df.loc[job_index, 'skills']
        resume_skills = resumes_df.loc[resume_index, 'skills']
        job_phrases = [sentence.strip() for sentence in (job_title + " " + job_description).split('.') if sentence.strip()]
        score = semantic_similarity_sbert_paraphrase_minilm_l6_v2(job_skills.split(",") + job_phrases, [resume_skills])
        matching_dataframe = matching_dataframe.append({
            'resume_id': resume_index,
            'job_id': job_index,
            'paraphrase-MiniLM-L6-v2_score': score
        }, ignore_index=True)

matching_dataframe.to_excel('matching_dataset3.xlsx', index=False)

In [ ]:
matching_dataframe

,resume_id,job_id,paraphrase-MiniLM-L6-v2_score
0,0.0,0.0,0.000
1,1.0,0.0,0.000
2,2.0,0.0,0.000
3,3.0,0.0,0.218
4,4.0,0.0,0.000
...,...,...,...
116,6.0,10.0,0.000
117,7.0,10.0,0.000
118,8.0,10.0,0.190
119,9.0,10.0,0.000


**bert-base-nli-mean-tokens**

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sbert_column = "bert-base-nli-mean-tokens_score"

# Initialiser le DataFrame pour stocker les résultats
columns = ['resume_id', 'job_id',sbert_column]
matching_dataframe = pd.DataFrame(columns=columns)

# Boucle sur chaque poste et chaque CV pour calculer la similarité sémantique
for job_index in tqdm(range(jobs_df.shape[0])):
    for resume_index in range(resumes_df.shape[0]):
        job_title = jobs_df.loc[job_index, 'job_title']
        job_description = jobs_df.loc[job_index, 'job_description']
        job_skills = jobs_df.loc[job_index, 'skills']
        resume_skills = resumes_df.loc[resume_index, 'skills']
        job_phrases = [sentence.strip() for sentence in (job_title + " " + job_description).split('.') if sentence.strip()]


        # Calculer la similarité sémantique avec SBERT
        skills_phrases = job_skills.split(",") + job_phrases
        skills_embeddings = sbert_model.encode(skills_phrases)
        resume_embeddings = sbert_model.encode([resume_skills])
        sbert_scores = cosine_similarity(skills_embeddings, resume_embeddings)
        sbert_score = max(sbert_scores)

        # Ajouter les scores au DataFrame de résultats
        matching_dataframe = matching_dataframe.append({
            'resume_id': resume_index,
            'job_id': job_index,
            sbert_column: sbert_score
        }, ignore_index=True)

# Enregistrer les résultats dans un fichier Excel
matching_dataframe.to_excel('matching_dataset_bert-base-nli-mean-tokens.xlsx', index=False)

In [ ]:
matching_dataframe

,resume_id,job_id,bert-base-nli-mean-tokens_score
0,0,0,[0.57029355]
1,1,0,[0.54126537]
2,2,0,[0.69288313]
3,3,0,[0.58560526]
4,4,0,[0.5266335]
...,...,...,...
116,6,10,[0.7841095]
117,7,10,[0.6796623]
118,8,10,[0.7613717]
119,9,10,[0.7861095]


 **ranking all-mpnet-base-v2_score**

In [ ]:

# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset.xlsx')

# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-mpnet-base-v2_score"

# Initialiser le DataFrame pour stocker les résultats de ranking
ranking_dataframe = pd.DataFrame(columns=['resume_id', 'job_id', sbert_column+'_rank'])

# Boucle sur chaque poste pour calculer le ranking des CVs
for job_index in range(jobs_df.shape[0]):
    # Extraire les scores de matching pour le poste courant
    matching_dataframe_one_job = matching_dataframe[matching_dataframe['job_id'] == job_index]

    # Calculer le ranking pour chaque colonne de similarité sémantique
    matching_dataframe_one_job[sbert_column+'_rank'] = matching_dataframe_one_job[sbert_column].rank(ascending=False).astype(int)

    # Ajouter les résultats de ranking au DataFrame de résultats
    ranking_dataframe = ranking_dataframe.append(matching_dataframe_one_job, ignore_index=True)

# Enregistrer les résultats de ranking dans un fichier Excel
ranking_dataframe.to_excel('ranking_base_v2.xlsx', index=False)

In [ ]:
ranking_dataframe

,resume_id,job_id,all-mpnet-base-v2_score_rank,all-mpnet-base-v2_score
0,0,0,6,0.000
1,1,0,6,0.000
2,2,0,6,0.000
3,3,0,1,0.123
4,4,0,6,0.000
...,...,...,...,...
116,6,10,8,0.000
117,7,10,8,0.000
118,8,10,4,0.090
119,9,10,8,0.000


  **ranking all-MiniLM-L12-v1**

In [ ]:

# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset1.xlsx')

# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-MiniLM-L12-v1_score"

# Initialiser le DataFrame pour stocker les résultats de ranking
ranking_dataframe = pd.DataFrame(columns=['resume_id', 'job_id', sbert_column+'_rank'])

# Boucle sur chaque poste pour calculer le ranking des CVs
for job_index in range(jobs_df.shape[0]):
    # Extraire les scores de matching pour le poste courant
    matching_dataframe_one_job = matching_dataframe[matching_dataframe['job_id'] == job_index]

    # Calculer le ranking pour chaque colonne de similarité sémantique
    matching_dataframe_one_job[sbert_column+'_rank'] = matching_dataframe_one_job[sbert_column].rank(ascending=False).astype(int)

    # Ajouter les résultats de ranking au DataFrame de résultats
    ranking_dataframe = ranking_dataframe.append(matching_dataframe_one_job, ignore_index=True)

# Enregistrer les résultats de ranking dans un fichier Excel
ranking_dataframe.to_excel('ranking_L12_v1.xlsx', index=False)

In [ ]:
ranking_dataframe

,resume_id,job_id,all-MiniLM-L12-v1_score_rank,all-MiniLM-L12-v1_score
0,0,0,6,0.000
1,1,0,6,0.000
2,2,0,6,0.000
3,3,0,1,0.117
4,4,0,6,0.000
...,...,...,...,...
116,6,10,8,0.000
117,7,10,8,0.000
118,8,10,1,0.256
119,9,10,8,0.000


 **ranking all-roberta-large-v1**

In [ ]:

# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset2.xlsx')

# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-roberta-large-v1_score"

# Initialiser le DataFrame pour stocker les résultats de ranking
ranking_dataframe = pd.DataFrame(columns=['resume_id', 'job_id', sbert_column+'_rank'])

# Boucle sur chaque poste pour calculer le ranking des CVs
for job_index in range(jobs_df.shape[0]):
    # Extraire les scores de matching pour le poste courant
    matching_dataframe_one_job = matching_dataframe[matching_dataframe['job_id'] == job_index]

    # Calculer le ranking pour chaque colonne de similarité sémantique
    matching_dataframe_one_job[sbert_column+'_rank'] = matching_dataframe_one_job[sbert_column].rank(ascending=False).astype(int)

    # Ajouter les résultats de ranking au DataFrame de résultats
    ranking_dataframe = ranking_dataframe.append(matching_dataframe_one_job, ignore_index=True)

# Enregistrer les résultats de ranking dans un fichier Excel
ranking_dataframe.to_excel('ranking_dataset_all-roberta-large-v1.xlsx', index=False)

In [ ]:
ranking_dataframe

,resume_id,job_id,all-roberta-large-v1_score_rank,all-roberta-large-v1_score
0,0,0,6,0.000
1,1,0,6,0.000
2,2,0,6,0.000
3,3,0,1,0.245
4,4,0,6,0.000
...,...,...,...,...
116,6,10,9,0.000
117,7,10,5,0.169
118,8,10,1,0.370
119,9,10,9,0.000


 **ranking paraphrase-MiniLM-L6-v2_score**

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset3.xlsx')

# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "paraphrase-MiniLM-L6-v2_score"

# Initialiser le DataFrame pour stocker les résultats de ranking
ranking_dataframe = pd.DataFrame(columns=['resume_id', 'job_id', sbert_column+'_rank'])

# Boucle sur chaque poste pour calculer le ranking des CVs
for job_index in range(jobs_df.shape[0]):
    # Extraire les scores de matching pour le poste courant
    matching_dataframe_one_job = matching_dataframe[matching_dataframe['job_id'] == job_index]

    # Calculer le ranking pour chaque colonne de similarité sémantique
    matching_dataframe_one_job[sbert_column+'_rank'] = matching_dataframe_one_job[sbert_column].rank(ascending=False).astype(int)

    # Ajouter les résultats de ranking au DataFrame de résultats
    ranking_dataframe = ranking_dataframe.append(matching_dataframe_one_job, ignore_index=True)

# Enregistrer les résultats de ranking dans un fichier Excel
ranking_dataframe.to_excel('ranking_L6_v2.xlsx', index=False)

In [ ]:
ranking_dataframe

,resume_id,job_id,paraphrase-MiniLM-L6-v2_score_rank,paraphrase-MiniLM-L6-v2_score
0,0,0,6,0.000
1,1,0,6,0.000
2,2,0,6,0.000
3,3,0,1,0.218
4,4,0,6,0.000
...,...,...,...,...
116,6,10,7,0.000
117,7,10,7,0.000
118,8,10,1,0.190
119,9,10,7,0.000


 **ranking base-nli-mean-tokens_score**

In [ ]:

# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset_bert-base-nli-mean-tokens.xlsx')

# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "bert-base-nli-mean-tokens_score"

# Initialiser le DataFrame pour stocker les résultats de ranking
ranking_dataframe = pd.DataFrame(columns=['resume_id', 'job_id', sbert_column+'_rank'])

# Boucle sur chaque poste pour calculer le ranking des CVs
for job_index in range(jobs_df.shape[0]):
    # Extraire les scores de matching pour le poste courant
    matching_dataframe_one_job = matching_dataframe[matching_dataframe['job_id'] == job_index]

    # Calculer le ranking pour chaque colonne de similarité sémantique
    matching_dataframe_one_job[sbert_column+'_rank'] = matching_dataframe_one_job[sbert_column].rank(ascending=False).astype(int)

    # Ajouter les résultats de ranking au DataFrame de résultats
    ranking_dataframe = ranking_dataframe.append(matching_dataframe_one_job, ignore_index=True)

# Enregistrer les résultats de ranking dans un fichier Excel
ranking_dataframe.to_excel('ranking_bert_tokens.xlsx', index=False)

In [ ]:
ranking_dataframe

,resume_id,job_id,bert-base-nli-mean-tokens_score_rank,bert-base-nli-mean-tokens_score
0,0,0,3,[0.57029355]
1,1,0,5,[0.54126537]
2,2,0,1,[0.69288313]
3,3,0,2,[0.58560526]
4,4,0,7,[0.5266335]
...,...,...,...,...
116,6,10,3,[0.7841095]
117,7,10,7,[0.6796623]
118,8,10,4,[0.7613717]
119,9,10,2,[0.7861095]


In [ ]:
# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset.xlsx')
# Charger les résultats de ranking
ranking_dataframe = pd.read_excel('ranking_base_v2.xlsx')
# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-mpnet-base-v2_score"
# Initialiser le DataFrame pour stocker les résultats de précision à k
columns=['job_id', 'all-mpnet-base-v2_precision']
precision_dataframe = pd.DataFrame(columns=columns)
# Boucle sur chaque poste pour calculer la précision à k
for job_index in range(jobs.shape[0]):
    # Calculer la précision à k pour le modèle all-mpnet-base-v2_precision
    k = 5
    top_k_ranked = ranking_dataframe[(ranking_dataframe['job_id'] == job_index) & (ranking_dataframe[sbert_column+'_rank'] <= k)]['resume_id'].tolist()
    precision = len(top_k_ranked) / k

    # Ajouter les résultats de précision à k au DataFrame de résultats
    precision_dataframe = precision_dataframe.append({'job_id': job_index, 'all-mpnet-base-v2_precision': precision}, ignore_index=True)

# Enregistrer les résultats de précision à k dans un fichier Excel
precision_dataframe.to_excel('precision_base_v2.xlsx', index=False)

In [ ]:
precision_dataframe

,job_id,all-mpnet-base-v2_precision
0,0.0,0.2
1,1.0,1.0
2,2.0,0.6
3,3.0,1.0
4,4.0,0.8
5,5.0,1.0
6,6.0,1.0
7,7.0,0.0
8,8.0,0.4
9,9.0,0.4


In [ ]:
# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset1.xlsx')
# Charger les résultats de ranking
ranking_dataframe = pd.read_excel('ranking_L12_v1.xlsx')
# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-MiniLM-L12-v1_score"
# Initialiser le DataFrame pour stocker les résultats de précision à k
columns=['job_id', 'all-MiniLM-L12-v1_precision']
precision_dataframe = pd.DataFrame(columns=columns)
# Boucle sur chaque poste pour calculer la précision à k
for job_index in range(jobs.shape[0]):
    # Calculer la précision à k pour le modèle all-MiniLM-L12-v1
    k = 5
    top_k_ranked = ranking_dataframe[(ranking_dataframe['job_id'] == job_index) & (ranking_dataframe[sbert_column+'_rank'] <= k)]['resume_id'].tolist()
    precision = len(top_k_ranked) / k

    # Ajouter les résultats de précision à k au DataFrame de résultats
    precision_dataframe = precision_dataframe.append({'job_id': job_index, 'all-MiniLM-L12-v1_precision': precision}, ignore_index=True)

# Enregistrer les résultats de précision à k dans un fichier Excel
precision_dataframe.to_excel('precision_all-MiniLM-L12-v1.xlsx', index=False)

In [ ]:
precision_dataframe

,job_id,all-MiniLM-L12-v1_precision
0,0.0,0.2
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
5,5.0,1.0
6,6.0,0.4
7,7.0,0.2
8,8.0,0.2
9,9.0,0.4


In [ ]:
# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset2.xlsx')
# Charger les résultats de ranking
ranking_dataframe = pd.read_excel('ranking_dataset_all-roberta-large-v1.xlsx')
# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "all-roberta-large-v1_score"
# Initialiser le DataFrame pour stocker les résultats de précision à k
columns=['job_id', 'all-roberta-large-v1_precision']
precision_dataframe = pd.DataFrame(columns=columns)
# Boucle sur chaque poste pour calculer la précision à k
for job_index in range(jobs.shape[0]):
    # Calculer la précision à k pour le modèle all-roberta-large-v1
    k = 5
    top_k_ranked = ranking_dataframe[(ranking_dataframe['job_id'] == job_index) & (ranking_dataframe[sbert_column+'_rank'] <= k)]['resume_id'].tolist()
    precision = len(top_k_ranked) / k

    # Ajouter les résultats de précision à k au DataFrame de résultats
    precision_dataframe = precision_dataframe.append({'job_id': job_index, 'all-roberta-large-v1_precision': precision}, ignore_index=True)

# Enregistrer les résultats de précision à k dans un fichier Excel
precision_dataframe.to_excel('precision_all-roberta-large-v1.xlsx', index=False)

In [ ]:
precision_dataframe

,job_id,all-roberta-large-v1_precision
0,0.0,0.2
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
5,5.0,1.0
6,6.0,0.8
7,7.0,0.2
8,8.0,0.2
9,9.0,0.4


In [ ]:
# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset3.xlsx')
# Charger les résultats de ranking
ranking_dataframe = pd.read_excel('ranking_L6_v2.xlsx')
# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "paraphrase-MiniLM-L6-v2_score"
# Initialiser le DataFrame pour stocker les résultats de précision à k
columns=['job_id', 'paraphrase-MiniLM-L6-v2_precision']
precision_dataframe = pd.DataFrame(columns=columns)
# Boucle sur chaque poste pour calculer la précision à k
for job_index in range(jobs.shape[0]):
    # Calculer la précision à k pour le modèle paraphrase-MiniLM-L6-v2
    k = 5
    top_k_ranked = ranking_dataframe[(ranking_dataframe['job_id'] == job_index) & (ranking_dataframe[sbert_column+'_rank'] <= k)]['resume_id'].tolist()
    precision = len(top_k_ranked) / k

    # Ajouter les résultats de précision à k au DataFrame de résultats
    precision_dataframe = precision_dataframe.append({'job_id': job_index, 'paraphrase-MiniLM-L6-v2_precision': precision}, ignore_index=True)

# Enregistrer les résultats de précision à k dans un fichier Excel
precision_dataframe.to_excel('precision_paraphrase-MiniLM-L6-v2.xlsx', index=False)

<ipython-input-80-0cb223c3e40a>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'paraphrase-MiniLM-L6-v2_precision': precision}, ignore_index=True)
<ipython-input-80-0cb223c3e40a>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'paraphrase-MiniLM-L6-v2_precision': precision}, ignore_index=True)
<ipython-input-80-0cb223c3e40a>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'paraphrase-MiniLM-L6-v2_precision': precision}, ignore_index=True)
<ipython-input-80-0cb223c3e40a>:18: FutureWarning: The frame.append me

In [ ]:
precision_dataframe

,job_id,all-roberta-large-v1_precision
0,0.0,0.2
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
5,5.0,1.0
6,6.0,0.8
7,7.0,0.2
8,8.0,0.2
9,9.0,0.4


In [ ]:
# Charger les résultats de matching
matching_dataframe = pd.read_excel('matching_dataset_bert-base-nli-mean-tokens.xlsx')
# Charger les résultats de ranking
ranking_dataframe = pd.read_excel('ranking_bert_tokens.xlsx')
# Définir les noms de colonnes pour les scores de similarité sémantique
sbert_column = "bert-base-nli-mean-tokens_score"
# Initialiser le DataFrame pour stocker les résultats de précision à k
columns=['job_id', 'bert-base-nli-mean-tokens_precision']
precision_dataframe = pd.DataFrame(columns=columns)
# Boucle sur chaque poste pour calculer la précision à k
for job_index in range(jobs.shape[0]):
    # Calculer la précision à k pour le modèle bert-base-nli-mean-tokens
    k = 5
    top_k_ranked = ranking_dataframe[(ranking_dataframe['job_id'] == job_index) & (ranking_dataframe[sbert_column+'_rank'] <= k)]['resume_id'].tolist()
    precision = len(top_k_ranked) / k

    # Ajouter les résultats de précision à k au DataFrame de résultats
    precision_dataframe = precision_dataframe.append({'job_id': job_index, 'bert-base-nli-mean-tokens_precision': precision}, ignore_index=True)

# Enregistrer les résultats de précision à k dans un fichier Excel
precision_dataframe.to_excel('precision_bert-base-nli-mean-tokens.xlsx', index=False)

<ipython-input-84-c8a4438dff91>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'bert-base-nli-mean-tokens_precision': precision}, ignore_index=True)
<ipython-input-84-c8a4438dff91>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'bert-base-nli-mean-tokens_precision': precision}, ignore_index=True)
<ipython-input-84-c8a4438dff91>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision_dataframe = precision_dataframe.append({'job_id': job_index, 'bert-base-nli-mean-tokens_precision': precision}, ignore_index=True)
<ipython-input-84-c8a4438dff91>:18: FutureWarning: The frame.app

In [ ]:
precision_dataframe

,job_id,bert-base-nli-mean-tokens_precision
0,0.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
5,5.0,1.0
6,6.0,1.0
7,7.0,1.0
8,8.0,1.0
9,9.0,1.0


In [ ]:
# Charger les résultats de précision à k pour chaque modèle de similarité sémantique
precision_dataframe_mpnet = pd.read_excel('precision_base_v2.xlsx')
precision_dataframe_miniLM = pd.read_excel('precision_all-MiniLM-L12-v1.xlsx')
precision_dataframe_roberta = pd.read_excel('precision_all-roberta-large-v1.xlsx')
precision_dataframe_paraphrase = pd.read_excel('precision_paraphrase-MiniLM-L6-v2.xlsx')
precision_dataframe_bert = pd.read_excel('precision_bert-base-nli-mean-tokens.xlsx')

# Calculer la précision moyenne pour chaque modèle et les ajouter au DataFrame de résultats
results = pd.concat([
    pd.DataFrame({'mpnet_base_v2_precision': [round(precision_dataframe_mpnet['all-mpnet-base-v2_precision'].mean(), 3)]}),
    pd.DataFrame({'paraphrase_MiniLM_L6_v2_precision': [round(precision_dataframe_paraphrase['paraphrase-MiniLM-L6-v2_precision'].mean(), 3)]}),
    pd.DataFrame({'MiniLM-L12-v1_precision': [round(precision_dataframe_miniLM['all-MiniLM-L12-v1_precision'].mean(), 3)]}),
    pd.DataFrame({'roberta_large_v1_precision': [round(precision_dataframe_roberta['all-roberta-large-v1_precision'].mean(), 3)]}),
    pd.DataFrame({'bert_base_nli_mean_tokens_precision': [round(precision_dataframe_bert['bert-base-nli-mean-tokens_precision'].mean(), 3)]})
], axis=1)

# Afficher le tableau de résultats finaux
print(results)

   mpnet_base_v2_precision  paraphrase_MiniLM_L6_v2_precision  \
0                    0.655                              0.382   

   MiniLM-L12-v1_precision  roberta_large_v1_precision  \
0                    0.655                       0.709   

   bert_base_nli_mean_tokens_precision  
0                                  1.0  
